In [ ]:
# default_exp uniformis

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# Uniform IS

## Basic Settings

In [ ]:
# imports
from bfh_mt_hs2020_sec_data.core import * 
from pathlib import Path
from typing import List, Tuple, Union, Set
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

In [ ]:
all_pivot_selected_folder  = "D:/data/parq_pivot_select"
all_pivoted_folder = "D:/data/parq_pivot_split"
all_processed_folder = "D:/data/parq_processed/"

col_list =    ["stmt","cik","ticker", "adsh","period","filed", "form","tag","value","report", "line", "fp", "uom"]
pivot_group = ["cik","ticker","adsh","form","period","filed","fp", "qtrs"]

In [ ]:
# init Spark
spark = get_spark_session() # Session anlegen
spark # display the moste important information of the session

## 00_Raw_data

In [ ]:
# loading the complete unpivoted dataset - if it is needed for debbuging
df_all_selected = spark.read.parquet(all_pivot_selected_folder).cache()

In [ ]:
# it happens sometimes, that the data could not be associated with a right sheet (bs, is, cf, ..). in this cases, the data can appea under "UN"
# so if expected information cannot be found in the appropriate statement, we have to look in the un statement
un_pivot_value = load_data(all_pivoted_folder, spark, "UN", "value")
un_pivot_pd = un_pivot_value.toPandas()

In [ ]:
un_pivot_pd.shape

(5989, 1774)

In [ ]:
def prepare_un_values(df_to_merge_into, attr_list):
    # add possible columns from un set to  cf data with prefix cpy_
    attributes = pivot_group[:] # create copy
    attributes.extend(attr_list)

    un_prepared = un_pivot_pd[attributes].copy()
    un_prepared.rename(columns=lambda x: x  if x in pivot_group else ("cpy_" + x), inplace=True)

    return pd.merge(df_to_merge_into, un_prepared, how='left', on=pivot_group)